<a href="https://colab.research.google.com/github/skhabiri/DS-Unit-2-Linear-Models/blob/master/module4-logistic-regression/skhabiri_LS_DS_214_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 4*

---

# Logistic Regression


## Assignment 🌯

You'll use a [**dataset of 400+ burrito reviews**](https://srcole.github.io/100burritos/). How accurately can you predict whether a burrito is rated 'Great'?

> We have developed a 10-dimensional system for rating the burritos in San Diego. ... Generate models for what makes a burrito great and investigate correlations in its dimensions.

- [ ] Do train/validate/test split. Train on reviews from 2016 & earlier. Validate on 2017. Test on 2018 & later.
- [ ] Begin with baselines for classification.
- [ ] Use scikit-learn for logistic regression.
- [ ] Get your model's validation accuracy. (Multiple times if you try multiple iterations.)
- [ ] Get your model's test accuracy. (One time, at the end.)
- [ ] Commit your notebook to your fork of the GitHub repo.


## Stretch Goals

- [ ] Add your own stretch goal(s) !
- [ ] Make exploratory visualizations.
- [ ] Do one-hot encoding.
- [ ] Do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html).
- [ ] Get and plot your coefficients.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

In [1]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Linear-Models/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [2]:
# Load data downloaded from https://srcole.github.io/100burritos/
import pandas as pd
df = pd.read_csv(DATA_PATH+'burritos/burritos.csv')

In [4]:
print(df.shape)
df.head()

(423, 66)


,Location,Burrito,Date,Neighborhood,Address,URL,Yelp,Google,Chips,Cost,Hunger,Mass (g),Density (g/mL),Length,Circum,Volume,Tortilla,Temp,Meat,Fillings,Meat:filling,Uniformity,Salsa,Synergy,Wrap,overall,Rec,Reviewer,Notes,Unreliable,NonSD,Beef,Pico,Guac,Cheese,Fries,Sour cream,Pork,Chicken,Shrimp,Fish,Rice,Beans,Lettuce,Tomato,Bell peper,Carrots,Cabbage,Sauce,Salsa.1,Cilantro,Onion,Taquito,Pineapple,Ham,Chile relleno,Nopales,Lobster,Queso,Egg,Mushroom,Bacon,Sushi,Avocado,Corn,Zucchini
0,Donato's taco shop,California,1/18/2016,Miramar,6780 Miramar Rd,http://donatostacoshop.net/,3.5,4.2,NaN,6.49,3.0,NaN,NaN,NaN,NaN,NaN,3.0,5.0,3.0,3.5,4.0,4.0,4.0,4.0,4.0,3.80,NaN,Scott,good fries: 4/5,NaN,NaN,x,x,x,x,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Oscar's Mexican food,California,1/24/2016,San Marcos,225 S Rancho Santa Fe Rd,http://www.yelp.com/biz/oscars-mexican-food-sa...,3.5,3.3,NaN,5.45,3.5,NaN,NaN,NaN,NaN,NaN,2.0,3.5,2.5,2.5,2.0,4.0,3.5,2.5,5.0,3.00,NaN,Scott,Fries: 3/5; too little meat,NaN,NaN,x,x,x,x,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Oscar's Mexican food,Carnitas,1/24/2016,NaN,NaN,NaN,NaN,NaN,NaN,4.85,1.5,NaN,NaN,NaN,NaN,NaN,3.0,2.0,2.5,3.0,4.5,4.0,3.0,3.0,5.0,3.00,NaN,Emily,NaN,NaN,NaN,NaN,x,x,NaN,NaN,NaN,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Oscar's Mexican food,Carne asada,1/24/2016,NaN,NaN,NaN,NaN,NaN,NaN,5.25,2.0,NaN,NaN,NaN,NaN,NaN,3.0,2.0,3.5,3.0,4.0,5.0,4.0,4.0,5.0,3.75,NaN,Ricardo,Go to average burrito place like Rigoberto's i...,NaN,NaN,x,x,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Pollos Maria,California,1/27/2016,Carlsbad,3055 Harding St,http://pollosmaria.com/,4.0,3.8,x,6.59,4.0,NaN,NaN,NaN,NaN,NaN,4.0,5.0,4.0,3.5,4.5,5.0,2.5,4.5,4.0,4.20,NaN,Scott,NaN,NaN,NaN,x,x,NaN,x,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
# check number of ns in class label
df["overall"].value_counts(dropna=False)

4.00000    74
3.50000    46
3.00000    39
4.50000    24
4.20000    24
2.00000    19
3.90000    17
3.70000    15
3.80000    15
4.10000    12
5.00000    11
3.20000    11
2.50000     9
3.60000     9
4.25000     8
4.80000     7
4.30000     7
3.10000     6
3.75000     6
2.60000     6
2.80000     5
4.60000     4
4.70000     4
2.70000     4
2.90000     4
3.30000     4
2.20000     4
4.40000     4
1.50000     4
3.40000     3
4.90000     3
NaN         2
3.25000     2
1.80000     2
2.40000     2
2.75000     2
2.30000     2
2.66000     1
1.00000     1
3.33333     1
Name: overall, dtype: int64

In [16]:
# Derive binary classification target:
# We define a 'Great' burrito as having an
# overall rating of 4 or higher, on a 5 point scale.
# Drop unrated burritos.
df = df.dropna(subset=['overall'])
df['Great'] = df['overall'] >= 4

In [17]:
df.shape

(421, 67)

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 421 entries, 0 to 422
Data columns (total 67 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Location        421 non-null    object 
 1   Burrito         421 non-null    object 
 2   Date            421 non-null    object 
 3   Neighborhood    92 non-null     object 
 4   Address         88 non-null     object 
 5   URL             87 non-null     object 
 6   Yelp            87 non-null     float64
 7   Google          87 non-null     float64
 8   Chips           26 non-null     object 
 9   Cost            414 non-null    float64
 10  Hunger          418 non-null    float64
 11  Mass (g)        22 non-null     float64
 12  Density (g/mL)  22 non-null     float64
 13  Length          283 non-null    float64
 14  Circum          281 non-null    float64
 15  Volume          281 non-null    float64
 16  Tortilla        421 non-null    float64
 17  Temp            401 non-null    flo

In [22]:
df["Burrito"].value_counts()

California            101
Carne asada            29
California             26
Carnitas               23
Local                  14
                     ... 
California Burrito      1
Tejano                  1
Hawaiian                1
Barbacoa                1
Especial                1
Name: Burrito, Length: 132, dtype: int64

In [23]:
# Clean/combine the Burrito categories
df['Burrito'] = df['Burrito'].str.lower()

# california is a boolean pd.Series
california = df['Burrito'].str.contains('california')
asada = df['Burrito'].str.contains('asada')
surf = df['Burrito'].str.contains('surf')
carnitas = df['Burrito'].str.contains('carnitas')

df.loc[california, 'Burrito'] = 'California'
df.loc[asada, 'Burrito'] = 'Asada'
df.loc[surf, 'Burrito'] = 'Surf & Turf'
df.loc[carnitas, 'Burrito'] = 'Carnitas'
df.loc[~california & ~asada & ~surf & ~carnitas, 'Burrito'] = 'Other'

In [26]:
california.head()

0     True
1     True
2    False
3    False
4     True
Name: Burrito, dtype: bool

In [27]:
df["Burrito"].value_counts()

California     169
Other          156
Asada           43
Surf & Turf     28
Carnitas        25
Name: Burrito, dtype: int64

In [28]:
# Drop some high cardinality categoricals
df = df.drop(columns=['Notes', 'Location', 'Reviewer', 'Address', 'URL', 'Neighborhood'])

In [29]:
# Drop some columns to prevent "leakage"
# Recommended and overal are derivatives of class label Great, causing leakage
df = df.drop(columns=['Rec', 'overall'])

In [30]:
df.shape

(421, 59)

In [33]:
df['Date'] = pd.to_datetime(df['Date'], infer_datetime_format=True)

In [58]:
df["Date"].value_counts()

2016-08-30    29
2016-06-24     9
2019-08-27     9
2016-05-13     7
2016-04-15     7
              ..
2016-09-13     1
2016-11-14     1
2016-12-15     1
2016-10-07     1
2019-08-24     1
Name: Date, Length: 169, dtype: int64

In [60]:
mask_2017 = pd.to_datetime('01-01-2017')
mask_2018 = pd.to_datetime('01-01-2018')


In [63]:
# split data into three categories
train = df[df["Date"] < mask_2017]
valid = df[(df["Date"] >= mask_2017) & (df["Date"] < mask_2018)]
test = df[df["Date"] >= mask_2018]

In [67]:
# Chck if the split left anything out or overlaps
assert len(df) == len(train) + len(valid) + len(test)

* X and Y label

In [68]:
train.shape

(298, 59)

In [72]:
X_train = train
y_train = X_train.pop("Great")

X_valid = valid
y_valid = X_valid.pop("Great")

X_test = test
y_test = X_test.pop("Great")

* Baseline model

In [75]:
y_train.unique()

array([False,  True])

In [76]:
y_train.value_counts(normalize=True)

False    0.590604
True     0.409396
Name: Great, dtype: float64

* Base line model: False
and that's accurate with 59% chance

* Logistic Regression

In [77]:
# # Import my predictor
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.impute import SimpleImputer

In [91]:
X_train.head(5)

,Burrito,Date,Yelp,Google,Chips,Cost,Hunger,Mass (g),Density (g/mL),Length,Circum,Volume,Tortilla,Temp,Meat,Fillings,Meat:filling,Uniformity,Salsa,Synergy,Wrap,Unreliable,NonSD,Beef,Pico,Guac,Cheese,Fries,Sour cream,Pork,Chicken,Shrimp,Fish,Rice,Beans,Lettuce,Tomato,Bell peper,Carrots,Cabbage,Sauce,Salsa.1,Cilantro,Onion,Taquito,Pineapple,Ham,Chile relleno,Nopales,Lobster,Queso,Egg,Mushroom,Bacon,Sushi,Avocado,Corn,Zucchini
0,California,2016-01-18,3.5,4.2,NaN,6.49,3.0,NaN,NaN,NaN,NaN,NaN,3.0,5.0,3.0,3.5,4.0,4.0,4.0,4.0,4.0,NaN,NaN,x,x,x,x,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,California,2016-01-24,3.5,3.3,NaN,5.45,3.5,NaN,NaN,NaN,NaN,NaN,2.0,3.5,2.5,2.5,2.0,4.0,3.5,2.5,5.0,NaN,NaN,x,x,x,x,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Carnitas,2016-01-24,NaN,NaN,NaN,4.85,1.5,NaN,NaN,NaN,NaN,NaN,3.0,2.0,2.5,3.0,4.5,4.0,3.0,3.0,5.0,NaN,NaN,NaN,x,x,NaN,NaN,NaN,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Asada,2016-01-24,NaN,NaN,NaN,5.25,2.0,NaN,NaN,NaN,NaN,NaN,3.0,2.0,3.5,3.0,4.0,5.0,4.0,4.0,5.0,NaN,NaN,x,x,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,California,2016-01-27,4.0,3.8,x,6.59,4.0,NaN,NaN,NaN,NaN,NaN,4.0,5.0,4.0,3.5,4.5,5.0,2.5,4.5,4.0,NaN,NaN,x,x,NaN,x,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [89]:
# Filter out nopn-numeric features
features = X_train.select_dtypes(include='number').columns
type(features)

pandas.core.indexes.base.Index

In [88]:
lin_model = LinearRegression()

In [106]:

# Fill na with SimpleImputer, we won't do that for y though
si = SimpleImputer()
X_train_sel = si.fit_transform(X_train[features])
X_valid_sel = si.fit_transform(X_valid[features])

In [153]:
print(X_train.shape)
print(X_valid.shape)

print(X_train[features].shape)
print(X_valid[features].shape)

print(X_train_sel.shape)
print(X_valid_sel.shape)

(298, 58)
(85, 58)
(298, 19)
(85, 19)
(298, 16)
(85, 18)


* Seems like columns of X_valid and X_train independantly are dropped by SimpleImputer. Perhaps there were all NaN

In [170]:
# Number of columns with all NaN
print((X_valid[features].isna().sum(axis=0)==len(X_valid)).sum())
print((X_train[features].isna().sum(axis=0)==len(X_train)).sum())

1
3


* Let's align columns of X_train_sel and X_valid_sel

In [182]:
(X_valid[features].isna().sum(axis=0)> 0.8*len(X_valid)).sum()

3

In [175]:
X_valid[features].shape

(85, 19)

In [202]:
Col_valid_na = X_valid[features].columns[(X_valid[features].isna().sum(axis=0) >= 0.7*len(X_valid))]
Col_train_na = X_train[features].columns[(X_train[features].isna().sum(axis=0) >= 0.2*len(X_train))]
print("Training columns to drop", Col_train_na)
print("validation columns to drop", Col_valid_na)
Col_valid_na_ser = pd.Series(Col_valid_na)
Col_train_na_ser = pd.Series(Col_train_na)

# Col_drop = pd.merge(Col_valid_na_ser, Col_train_na_ser, how='outer')

Training columns to drop Index(['Yelp', 'Google', 'Mass (g)', 'Density (g/mL)', 'Length', 'Circum',
       'Volume', 'Queso'],
      dtype='object')
validation columns to drop Index(['Yelp', 'Google', 'Mass (g)', 'Density (g/mL)', 'Queso'], dtype='object')


ValueError: ignored

In [ ]:
# df1 = df.dropna(thresh=370, axis=1)

In [162]:
if X_valid[features]
col_list = X_valid.columns
X_train[col_list].shape

(298, 58)

In [97]:
lin_model.fit(X_train_sel, y_train)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [149]:
X_valid_sel

array([[ 3.84615385,  4.35384615,  6.6       , ...,  1.5       ,
         3.5       ,  4.5       ],
       [ 3.84615385,  4.35384615,  6.6       , ...,  4.2       ,
         3.75      ,  5.        ],
       [ 3.84615385,  4.35384615,  8.5       , ...,  4.3       ,
         4.2       ,  5.        ],
       ...,
       [ 4.        ,  4.5       , 11.5       , ...,  3.5       ,
         4.        ,  2.        ],
       [ 3.84615385,  4.35384615,  7.89      , ...,  3.5       ,
         4.3       ,  4.5       ],
       [ 3.84615385,  4.35384615,  7.89      , ...,  5.        ,
         5.        ,  3.        ]])

In [140]:
X_valid[X_valid.columns == (X_valid.isna().sum(axis=0)==len(X_valid.columns))]

ValueError: ignored

In [100]:
# Apply our model to new data
lin_model.predict(X_train_sel)

array([ 0.44488067, -0.09326176,  0.03711416,  0.30425209,  0.78847568,
        0.48012925, -0.14529904, -0.00142295,  0.42455352, -0.22353051,
       -0.05458493,  0.02949579,  0.27050957,  0.1041616 ,  0.08296338,
        0.6553216 ,  0.81796274,  0.45778379,  0.46962221,  0.09064877,
        0.22460883,  0.32635558,  0.08296122,  0.62541771,  0.59759904,
        0.22264169,  0.16992311,  0.56473841,  0.73924418,  0.32094949,
        0.48235143, -0.01063053, -0.30003272, -0.90046533,  0.33739931,
        0.20862901,  0.57738671,  0.38728819,  0.60539483,  0.68782292,
        0.40553032,  0.2620027 ,  0.6384121 ,  0.35917695,  0.59154376,
        0.19163055,  0.3939565 ,  0.27142973,  0.08377925,  0.19528983,
        0.70002524,  0.91186785, -0.21679332,  0.55132464,  0.5167455 ,
        0.23981618,  0.53983593,  0.5638558 ,  0.82709064,  1.01352166,
        0.47791233,  0.26390009,  0.44651361,  0.07924928,  0.64161032,
        0.51677498,  0.47322073,  0.43331513,  1.05547855,  1.14

In [ ]:
# # Import my predictor
# from sklearn.linear_model import LinearRegression, LogisticRegression
# from sklearn.impute import SimpleImputer

# # Instantiate
# lin_model = LinearRegression()

# # Get your data in order
# features = ['Pclass', 'Age', 'Fare']
# X_train_small = X_train[features] 
# X_val_small = X_val[features]

# # Transform our data
# si = SimpleImputer()
# X_train_small_t = si.fit_transform(X_train_small)
# X_val_small_t = si.fit_transform(X_val_small)

# # Fit model
# lin_model.fit(X_train_small_t, y_train)

# # Apply our model to new data
# Y_val_pred = lin_model.predict(X_val_small_t)